# FCT Unesp - TCC Treino 1
Aluno: Pétrus Antonio Bárbosa Pradella

TCC2 - Treinando análise preditiva para dados de transações comerciais.

Teste 1
    - DataSet Utilizado: Weekly Sales Transactions
    - Fonte: Kaggle
    - Link: https://www.kaggle.com/crawford/weekly-sales-transactions
    
    Dataset disponibilizado pelo "UC Irvine Machine Learning Repository"

Caracteristicas gerais do Dataset:

    - Número de Instancias: 811
    - Número de Atributos: 53
    - Estatisticas de Uso em 29/08/2019 (kaggle):
        24,992 views
        3,112 downloads

    - Mais detalhes tecnicos podem ser encontrados em:
        https://archive.ics.uci.edu/ml/datasets/Sales_Transactions_Dataset_Weekly


### Importando as bibliotecas necessárias

In [4]:
import numpy as np
import pandas as pd
import math


from pandas import Series, DataFrame
from sklearn.model_selection import train_test_split



In [5]:
#Configurações para as análises algébricas:

autocorr_nlags_max = 13 # Maximum number of lags for calculating autocorrelation
limiar_de_correlacao = 0.7 # Use >= 0.7 for strongly correlated
param_top_perc = 0.05 #Porcentagem limiar para escolher os melhores produtos, se aumentar esse valor mais produtos serão consideradores como "produtos populares"

### Carregando a base de dados

In [ ]:
current_folder = "C:/Users/Petrus/Desktop/UNESP/Docs 2016-2020/2019/Segundo Semestre/TCC2/ProjetoTCC2/Treino1";
dataset_file = current_folder + '/Sales_Transactions_Dataset_Weekly.csv';
dataset = pd.read_csv(dataset_file)

In [ ]:
dataset.head(5)

### Filtrando os atributos Normalizados

In [ ]:
col_names = dataset.columns

print(col_names)

In [ ]:
#dataset_filtered = pd.read_csv(dataset_file).filter(regex='Product_Code|^W[0-9]+$')

#Tambem dá para fazer via definição direta... mas o regexa ali é bem mais facil quando se tem como validar ele em algum outro lugar.
dataset_filtered = pd.read_csv(dataset_file, 
                       usecols=[col for col in col_names if 'Product_Code' in col or 'W' in col],
                       index_col = 'Product_Code')

print(dataset_filtered.shape)
dataset_filtered.head(5)

In [ ]:
#n_produtos = dataset_filtered[['Product_Code']].count().sum();  # Dá erro pois Product_code virou um index_de_coluna e não mais uma coluna separada!

n_produtos = dataset_filtered.shape[0]; #Como fizemos a removeção da coluna de Product_code não dá mais para contar ela, contudo o número de produtos é o mesmo que o número de linhas!
n_semanas = len(dataset_filtered.columns);  #Se não tivesse removido a primeira coluna, deveria ser [nColumns - 1];

print("Número de Produtos Cadastrados: ", n_produtos)
print("Número de Semanas Contabilizadas: ", n_semanas)


### Aplicando lógica de regressão linear! (Ténicas usada "Spearman")

In [ ]:
#Aplicando o método Spearman
# Mais informações Tecnicas: https://kite.com/python/examples/665/scipy-compute-the-spearman-correlation
# Mais informações Pedagógicas: https://statistics.laerd.com/statistical-guides/spearmans-rank-order-correlation-statistical-guide.php

#A ténicas Spearman é recomendada para esse caso pois trata-se de um regressão linear para um dataset que possui apenas dois atributos a serem analisados.

correlacao_produtos = dataset_filtered.transpose().corr(method='spearman')
print(correlacao_produtos.shape)
correlacao_produtos.head(5)



### Procurando valores com coeficiente alto!


In [ ]:
set_de_coeficiente_alto = set() # record the individual Product_Code values involved
pares_de_coeficiente_alto = [] # record the Product_Code pairs and the correlation value

for row in correlacao_produtos:
    for col in correlacao_produtos.columns:
        if row < col:
            
            valor_de_correlacao = correlacao_produtos.loc[row, col]

            if abs(valor_de_correlacao) >= limiar_de_correlacao:
                # Se o valor é maior que o limiar, salvar ele!
                #print(row)
                #print(col)
                #print(corr_val)
                pares_de_coeficiente_alto.append((row, col, valor_de_correlacao))
                #Adiciona a linha e a coluna
                set_de_coeficiente_alto.add(row) 
                set_de_coeficiente_alto.add(col)
            
#print(high_corr_list)
# Convert list to a dataframe
high_correlation_dataframe = pd.DataFrame(pares_de_coeficiente_alto, columns=['Product_1', 'Product_2', 'Corr'])
print(f"Número de produtos com coeficientes de correlação >= {limiar_de_correlacao} ou <= -{limiar_de_correlacao}: {high_correlation_dataframe.shape[0]}")
print(f"Número total de produtos: {len(set_de_coeficiente_alto)}")
high_correlation_dataframe.head(5)

### Analisando popularidade dos itens (maior e menor venda)


In [ ]:
n_produtos = dataset_filtered.shape[0];
n_semanas = len(dataset_filtered.columns);

print("Número de Produtos Cadastrados: ", n_produtos)
print("Número de Semanas Contabilizadas: ", n_semanas)
print(' ')
print(' ')

num_top = math.floor(param_top_perc * n_produtos)
print(f'Encontrar os [{num_top}] produtos mais e menos populares com base na média das vendas semanáis:')

media_venda_mensal = dataset_filtered.mean(axis=1)

produtos_mais_vendidos = media_venda_mensal.nlargest(n=num_top).index.values.tolist()
produtos_menos_vendidos = media_venda_mensal.nsmallest(n=num_top).index.values.tolist()

dataset_mais_vendidos = dataset_filtered[dataset_filtered.index.isin(produtos_mais_vendidos)]
dataset_menos_vendidos = dataset_filtered[dataset_filtered.index.isin(produtos_menos_vendidos)]

In [2]:

print(f"Foram encontrados os [{num_top}] produtos mais populares: {dataset_mais_vendidos.shape[0]}")
dataset_mais_vendidos.head(5)

NameError: name 'num_top' is not defined

In [ ]:

print(f"Foram encontrados os [{num_top}] produtos menos populares: {dataset_menos_vendidos.shape[0]}")
dataset_menos_vendidos.head(5)